In [24]:
'''Step 1 - mount Google drive'''
from google.colab import drive
from pathlib import Path

drive.mount('/content/gdrive')
data_dir = Path('/content/gdrive/') / 'My Drive' / 'torch-scae' / 'data' / 'boneage'

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [17]:

!rm -rf ./torch-scae
!git clone https://github.com/karayanni/torch-scae
%cd ./torch-scae
!pip install -q .

Cloning into 'torch-scae'...
remote: Enumerating objects: 354, done.
remote: Counting objects: 100% (354/354), done.
remote: Compressing objects: 100% (210/210), done.
remote: Total 832 (delta 218), reused 272 (delta 143), pack-reused 478
Receiving objects: 100% (832/832), 3.44 MiB | 13.66 MiB/s, done.
Resolving deltas: 100% (538/538), done.
/content/torch-scae/torch-scae


In [18]:
%load_ext tensorboard

!mkdir /content/lightning_logs
#%tensorboard --logdir '/content/lightning_logs'

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard
mkdir: cannot create directory ‘/content/lightning_logs’: File exists


In [19]:
import pathlib

import numpy as np
import torch
import torchvision
from torch_scae_experiments.boneage.train import train
from torch_scae_experiments.boneage.hparams import model_params
from pytorch_lightning import seed_everything

seed_everything(42)

42

In [20]:
from pprint import pprint

pprint(model_params)

{'image_shape': (3, 256, 256),
 'n_classes': 1,
 'n_obj_caps': 16,
 'n_part_caps': 30,
 'ocae_decoder_capsule_params': {'allow_deformations': True,
                                 'caps_dropout_rate': 0.0,
                                 'dim_caps': 32,
                                 'hidden_sizes': (128,),
                                 'learn_vote_scale': True,
                                 'noise_scale': 4.0,
                                 'noise_type': 'uniform',
                                 'similarity_transform': False},
 'ocae_encoder_set_transformer_params': {'dim_hidden': 16,
                                         'dim_out': 256,
                                         'layer_norm': True,
                                         'n_heads': 1,
                                         'n_layers': 3},
 'pcae_cnn_encoder_params': {'activate_final': True,
                             'kernel_sizes': [3, 3, 3, 3],
                             'out_channels': [128, 

In [21]:
# Customize model parameters

# model_params.update(dict(
#     n_part_caps=36,
#     n_obj_caps=10,
#     pcae_template_generator_params=dict(
#         template_size=(11, 11),
#         template_nonlin='sigmoid',
#         colorize_templates=False,
#     ),
#     scae_params=dict(
#         posterior_sparsity_loss_type='entropy',
#         posterior_within_example_sparsity_weight=1,
#         posterior_between_example_sparsity_weight=1,
#     )
# ))

In [25]:
training_params = dict(
    #data_dir=str(pathlib.Path.home() / 'torch-scae/data/boneage'),
    data_dir = str(data_dir),
    gpus=1,
    batch_size=16,
    num_workers=2,
    max_epochs=100,
    learning_rate=3e-5,
    optimizer_type='RMSprop',
    use_lr_scheduler=True,
    lr_scheduler_decay_rate=0.997
)

In [27]:
torch.cuda.reset_max_memory_cached()
torch.cuda.reset_max_memory_allocated()
torch.cuda.reset_accumulated_memory_stats()

train(model_params, **training_params)

GPU available: True, used: True
No environment variable for node rank defined. Set as 0.
CUDA_VISIBLE_DEVICES: [0]

    | Name                                              | Type                      | Params
--------------------------------------------------------------------------------------------
0   | scae                                              | SCAE                      | 2 M   
1   | scae.part_encoder                                 | CapsuleImageEncoder       | 984 K 
2   | scae.part_encoder.encoder                         | CNNEncoder                | 446 K 
3   | scae.part_encoder.encoder.network                 | Sequential                | 446 K 
4   | scae.part_encoder.encoder.network.0               | Conv2d                    | 3 K   
5   | scae.part_encoder.encoder.network.1               | ReLU                      | 0     
6   | scae.part_encoder.encoder.network.2               | Conv2d                    | 147 K 
7   | scae.part_encoder.encoder.network.3      

{'prior': tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], device='cuda:0'), 'posterior': tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], device='cuda:0')}


/usr/local/lib/python3.6/dist-packages/pytorch_lightning/utilities/distributed.py:23: UserWarning: The dataloader, train dataloader, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` in the `DataLoader` init to improve performance.
  warnings.warn(*args, **kwargs)
/usr/local/lib/python3.6/dist-packages/pytorch_lightning/utilities/distributed.py:23: UserWarning: The dataloader, val dataloader 0, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` in the `DataLoader` init to improve performance.
  warnings.warn(*args, **kwargs)


RuntimeError: ignored

In [ ]:
# resume training
training_params.update(
    max_epochs=200,
    resume_from_checkpoint= '/content/lightning_logs/version_0/checkpoints/epoch=99.ckpt')
)
train(model_params, **training_params)